```{contents}
```
## Tool State

In LangGraph, **Tool State** is the portion of the shared execution state that **stores information about tool usage, tool results, and tool-related metadata** across nodes and across time.
It allows the graph to **reason over past actions**, coordinate multiple tools, enforce policies, and build long-horizon workflows.

---

### **1. Motivation: Why Tool State Exists**

LLM systems that call tools are not stateless pipelines.
They must remember:

* Which tools were called
* With what arguments
* What results were returned
* Whether a tool failed or succeeded
* What remains to be done

This memory of tool activity is exactly what **Tool State** provides.

---

### **2. Conceptual Role of Tool State**

```
LLM → Decide Tool → Call Tool → Observe Result → Update Tool State → Next Decision
```

Tool State becomes part of the **global graph state**, allowing future nodes and agents to reason about past tool interactions.

---

### **3. Structure of Tool State**

Tool State is user-defined and typically embedded inside the main state schema.

```python
class ToolRecord(TypedDict):
    name: str
    args: dict
    output: str
    success: bool

class State(TypedDict):
    messages: list
    tool_history: list[ToolRecord]
    next_action: str
```

This design makes tool behavior **explicit, inspectable, and persistent**.

---

### **4. How Tool State Is Updated**

Each tool node returns **partial state updates**.

```python
def weather_tool_node(state: State):
    result = call_weather_api(state["city"])
    
    record = {
        "name": "weather_api",
        "args": {"city": state["city"]},
        "output": result,
        "success": True
    }

    return {
        "tool_history": state["tool_history"] + [record]
    }
```

LangGraph automatically merges this update using the state reducer.

---

### **5. Using Tool State for Control Flow**

Tool State often drives **routing decisions**.

```python
def route(state: State):
    last = state["tool_history"][-1]

    if not last["success"]:
        return "retry_tool"
    if "error" in last["output"]:
        return "fallback_tool"
    return "continue"
```

This enables **self-healing pipelines**.

---

### **6. Tool State vs Conversation State**

| Aspect         | Tool State             | Conversation State |
| -------------- | ---------------------- | ------------------ |
| Purpose        | Track tool execution   | Track dialogue     |
| Content        | Calls, results, status | Messages           |
| Used for       | Control & planning     | Context for LLM    |
| Lifetime       | Long-lived             | Long-lived         |
| Drives routing | Yes                    | Indirect           |

Both live inside the same global state but serve distinct functions.

---

### **7. Advanced Patterns with Tool State**

#### **a. Tool Budgeting**

```python
if len(state["tool_history"]) > 10:
    raise StopExecution("Tool budget exceeded")
```

#### **b. Tool Verification Loop**

```
Call Tool → Verify Result → If invalid → Retry or Switch Tool
```

#### **c. Tool Arbitration (Multi-Agent)**

Agents examine tool history before choosing the next tool.

---

### **8. Production-Grade Considerations**

| Concern      | Strategy                         |
| ------------ | -------------------------------- |
| Auditing     | Persist tool_history to database |
| Security     | Sanitize stored tool inputs      |
| Cost control | Track tool invocation counts     |
| Reliability  | Store failures and retries       |
| Debugging    | Visualize tool timeline          |
| Compliance   | Immutable tool logs              |

---

### **9. Mental Model**

> **Tool State is the operational memory of your AI system.**

Without Tool State, agents are **stateless guessers**.
With Tool State, agents become **deliberate planners**.

---

### **10. Summary**

| Capability Enabled    | By Tool State |
| --------------------- | ------------- |
| Multi-step planning   | Yes           |
| Self-correction       | Yes           |
| Failure recovery      | Yes           |
| Autonomous control    | Yes           |
| Auditability          | Yes           |
| Enterprise compliance | Yes           |


### Demonstration

In [1]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END

class ToolRecord(TypedDict):
    name: str
    args: dict
    output: str
    success: bool

class State(TypedDict):
    query: str
    tool_history: List[ToolRecord]
    final_answer: str


In [2]:
def search_tool(query: str) -> str:
    if "fail" in query:
        return "ERROR: search failed"
    return f"Search results for '{query}'"

def tool_node(state: State):
    result = search_tool(state["query"])

    record = {
        "name": "search_tool",
        "args": {"query": state["query"]},
        "output": result,
        "success": not result.startswith("ERROR")
    }

    return {
        "tool_history": state["tool_history"] + [record]
    }

def router(state: State):
    last = state["tool_history"][-1]

    if not last["success"]:
        return "retry"
    return "final"

def retry_node(state: State):
    state["query"] = "recovered query"
    return state

def final_node(state: State):
    output = state["tool_history"][-1]["output"]
    return {"final_answer": f"Completed: {output}"}


In [3]:
builder = StateGraph(State)

builder.add_node("tool", tool_node)
builder.add_node("retry", retry_node)
builder.add_node("final", final_node)

builder.set_entry_point("tool")

builder.add_conditional_edges("tool", router, {
    "retry": "retry",
    "final": "final"
})

builder.add_edge("retry", "tool")
builder.add_edge("final", END)

graph = builder.compile()


In [4]:
result = graph.invoke({
    "query": "this will fail",
    "tool_history": [],
    "final_answer": ""
})

print(result["final_answer"])


Completed: Search results for 'recovered query'
